# Data preparation

In [1]:
!conda install --yes pytorch torchvision -c pytorch

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::holoviews==1.15.0=py39hecd8cb5_0
  - defaults/noarch::babel==2.9.1=pyhd3eb1b0_0
  - defaults/noarch::nbclassic==0.3.5=pyhd3eb1b0_0
  - defaults/noarch::cookiecutter==1.7.3=pyhd3eb1b0_0
  - defaults/osx-64::anaconda-project==0.11.1=py39hecd8cb5_0
  - defaults/osx-64::applaunchservices==0.3.0=py39hecd8cb5_0
  - defaults/noarch::pyls-spyder==0.4.0=pyhd3eb1b0_0
  - defaults/osx-64::jupyterlab==3.4.4=py39hecd8cb5_0
  - defaults/osx-64::datashader==0.14.1=py39hecd8cb5_0
  - defaults/osx-64::anaconda==2022.10=py39_0
  - defaults/osx-64::hvplot==0.8.0=py39hecd8cb5_0
  - defaults/osx-64::python-lsp-black==1.2.1=py39hecd8cb5_0
  - defaults/osx-64::bkcharts==0.2=py39hecd8cb5_1
  - defaults/osx-64::conda-build==3.22.0=py39hecd8cb5_0
  - defaults/osx-64::anaconda-navigator==2.3.2=py39hecd8cb5_0
  - defaults/osx-64::_ipyw_jlab_nb

ffmpeg-4.3           | 10.1 MB   | ##################################### | 100% 
_anaconda_depends-20 | 69 KB     | ##################################### | 100% 
nettle-3.7.3         | 380 KB    | ##################################### | 100% 
openssl-1.1.1t       | 3.3 MB    | ##################################### | 100% 
anaconda-custom      | 4 KB      | ##################################### | 100% 
pytorch-2.0.0        | 76.4 MB   | ##################################### | 100% 
pytz-2022.7          | 210 KB    | ##################################### | 100% 
certifi-2022.12.7    | 151 KB    | ##################################### | 100% 
libtasn1-4.19.0      | 67 KB     | ##################################### | 100% 
lame-3.100           | 316 KB    | ##################################### | 100% 
openh264-2.1.1       | 655 KB    | ##################################### | 100% 
gnutls-3.6.15        | 974 KB    | ##################################### | 100% 
jsonpointer-2.1      | 9 KB 

In [2]:
!pip install torchvision

In [4]:
import os
import numpy as np
np.random.seed(666)
import torchvision
import torchvision.datasets as datasets
if not os.path.isdir('data'):
    ! mkdir data

In [6]:
import torchvision.datasets as datasets
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)

In [7]:
x_train = np.array([np.array(img[0])/255. for img in mnist_trainset])
y_train = np.array([img[1] for img in mnist_trainset])
x_test = np.array([np.array(img[0])/255. for img in mnist_testset])
y_test = np.array([img[1] for img in mnist_testset])

np.savez('data/cifar10', arr_0=x_train, arr_1=y_train, arr_2=x_test, arr_3=y_test)

In [9]:
# load images as queries

dataset = np.load('data/cifar10.npz')
x_train = dataset['arr_0']
y_train = dataset['arr_1']
x_test = dataset['arr_2']
y_test = dataset['arr_3']

print("training data shape: {}, {}, testing data shape: {}, {}".format(x_train.shape, y_train.shape, x_test.shape, y_test.shape))

training data shape: (60000, 28, 28), (60000,), testing data shape: (10000, 28, 28), (10000,)


# Start Blacklight detection

In [10]:
!pip install config

In [11]:
#!pip install logging

In [12]:
from probabilistic_fingerprint import *

In [13]:
# parameters for blacklight MNSIT
window_size = 50
hash_kept = 50
roundto = 50
step_size = 1
workers = 5
threshold = 25

In [14]:
def get_tracker(query, window_size, hash_kept, roundto, step_size, workers):
    tracker = InputTracker(query, window_size, hash_kept, round=roundto, step_size=step_size, workers=workers)
    LOGGER.info("Blacklight detector created.")
    return tracker

In [15]:
# create a tracker as detector and detect malicious queries
tracker = get_tracker(x_train[0], window_size, hash_kept, roundto, step_size, workers)
match_list = []
id = 0
for query in x_test:
    match_num = tracker.add_img(query)
    match_list.append(match_num)
    if(match_num>threshold):
        LOGGER.info("Image: {}, max match: {}, attack_query: {}".format(id, match_num, match_num>threshold))
    id += 1

[13:12:46,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2128301390.py:3 INFO]: Blacklight detector created.
[13:12:47,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 597, max match: 27, attack_query: True
[13:12:48,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 642, max match: 28, attack_query: True
[13:12:48,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 717, max match: 27, attack_query: True
[13:12:48,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 1031, max match: 28, attack_query: True
[13:12:49,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 1345, max match: 29, attack_query: True
[13:12:50,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 1722

[13:13:20,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 8139, max match: 26, attack_query: True
[13:13:20,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 8158, max match: 26, attack_query: True
[13:13:20,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 8165, max match: 28, attack_query: True
[13:13:20,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 8183, max match: 26, attack_query: True
[13:13:20,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 8198, max match: 30, attack_query: True
[13:13:20,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py:9 INFO]: Image: 8225, max match: 26, attack_query: True
[13:13:21,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/2323176259.py

In [18]:
# calculate false positive rate
LOGGER.info("False positive rate on MNSIT test dataset: {}".format(np.sum([1 for i in match_list if i > threshold])/len(x_test)))

[13:14:34,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_30262/3485651310.py:2 INFO]: False positive rate on CiFAR-10 test dataset: 0.0086
